## Setup and Imports

In [237]:
import time
import numpy as np
import pandas as pd
import altair as alt
from meridian.model import model
from meridian.analysis import fast_response_curves
from meridian.analysis import visualizer

# Enable Altair for Jupyter
alt.data_transformers.enable('json')

print("✅ All imports successful!")

✅ All imports successful!


In [238]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the Model

In [239]:
# Load the real Audi model
print("Loading Audi model...")
mmm = model.load_mmm('meridian/data/0_test_Audi.pkl')

print(f"✅ Loaded Audi model successfully!")
print(f"Model details:")
print(f"  - Geos: {mmm.n_geos}")
print(f"  - Time periods: {mmm.n_times}")
print(f"  - Media channels: {mmm.n_media_channels}")
print(f"  - Channel names: {list(mmm.input_data.media_channel.values)}")
print(f"  - Posterior samples: {mmm.inference_data.posterior.chain.size} chains × {mmm.inference_data.posterior.draw.size} draws")
print(f"  - Has revenue per KPI: {mmm.input_data.revenue_per_kpi is not None}")

Loading Audi model...
✅ Loaded Audi model successfully!
Model details:
  - Geos: 30
  - Time periods: 78
  - Media channels: 3
  - Channel names: ['TV', 'Display', 'Video']
  - Posterior samples: 2 chains × 2000 draws
  - Has revenue per KPI: True


In [ ]:
# Initialize FastResponseCurves
print("Initializing FastResponseCurves...")
fast_rc = fast_response_curves.FastResponseCurves(mmm)

# Inspect extracted parameters
print("\n📊 Extracted Median Parameters:")
if fast_rc.has_media_data:
    print("Media Parameters:")
    for param_name, values in fast_rc.media_params.items():
        print(f"  {param_name}: {values}")

if fast_rc.has_rf_data:
    print("\nR&F Parameters:")
    for param_name, values in fast_rc.rf_params.items():
        print(f"  {param_name}: {values}")

print(f"\n🎯 Total channels: {fast_rc.n_total_channels}")
print(f"Channel names: {fast_rc.channel_names}")

Initializing FastResponseCurves...

📊 Extracted Median Parameters:
Media Parameters:
  ec: [1.7848499 2.4640331 6.6890507]
  slope: [1.9554161 1.6472095 2.2099836]
  alpha: [0.72785664 0.44213519 0.35649306]
  beta: [[0.80981284 1.9012511  0.08089066]
 [0.51529133 0.23066592 0.08046897]
 [1.3150611  0.1574071  0.08081126]
 [0.64918983 0.5938306  0.08068982]
 [1.1782352  0.23036027 0.08046756]
 [1.5945857  1.6312532  0.08092123]
 [2.1041825  0.14655419 0.08024985]
 [0.35678148 0.17647648 0.08053552]
 [0.62451607 0.3410471  0.08085941]
 [0.76727474 1.2669538  0.08080655]
 [1.9601594  0.6096046  0.08085322]
 [1.2308985  2.553587   0.08092396]
 [0.66949415 0.55078614 0.08105425]
 [0.5347257  0.73873764 0.08055339]
 [0.5783776  0.40243137 0.08093156]
 [0.5463298  0.15169615 0.0805738 ]
 [1.5988507  0.21528497 0.080392  ]
 [0.30869478 0.05210268 0.08059177]
 [0.6539594  0.34719568 0.08061485]
 [0.5910076  0.23953283 0.08035229]
 [0.7109065  0.52290195 0.08087035]
 [0.90783846 0.32954907 0.08

1. First, Get the Response Curve data for the original code

In [219]:
from meridian import constants as c

confidence_level: float = c.DEFAULT_CONFIDENCE_LEVEL
selected_times: frozenset[str] | None = None
by_reach: bool = True

# 1. Get the response curve data based on the original posterior samples
print("Getting actual response curves...")
media_effects = visualizer.MediaEffects(mmm)
total_num_channels = len(media_effects._meridian.input_data.get_all_channels())
num_channels_displayed = total_num_channels
actual_rc_data = media_effects._transform_response_curve_metrics(
    num_channels_displayed,
    confidence_level=confidence_level,
    selected_times=selected_times,
    by_reach=by_reach,
)

Getting actual response curves...


In [220]:

# 2. Get the fast response curve data using same granular multipliers
print("Initializing FastResponseCurves with granular multipliers...")
fast_rc = fast_response_curves.FastResponseCurves(mmm)
actual_multipliers = sorted(actual_rc_data['spend_multiplier'].unique())
fast_rc_data = fast_rc.response_curves_data(spend_multipliers=actual_multipliers).to_dataframe().reset_index()

Initializing FastResponseCurves with granular multipliers...


In [221]:
fast_rc_data

,channel,metric,spend_multiplier,spend,incremental_outcome,roi
0,TV,mean,0.00,0.000000e+00,0.000000e+00,0.000000
1,TV,mean,0.01,3.139351e+05,6.499977e+03,0.020705
2,TV,mean,0.02,6.278702e+05,2.519622e+04,0.040130
3,TV,mean,0.03,9.418053e+05,5.563460e+04,0.059072
4,TV,mean,0.04,1.255740e+06,9.754739e+04,0.077681
...,...,...,...,...,...,...
655,Video,mean,2.15,1.044173e+07,1.118925e+06,0.107159
656,Video,mean,2.16,1.049030e+07,1.127067e+06,0.107439
657,Video,mean,2.17,1.053886e+07,1.135214e+06,0.107717
658,Video,mean,2.18,1.058743e+07,1.143366e+06,0.107993


In [233]:
# Plot the response curves
step = 0.01
fast_rc.plot_response_curves(spend_multipliers=np.arange(0, 10+step, step=step), plot_separately=True)

alt.FacetChart(...)

In [232]:
media_effects = visualizer.MediaEffects(mmm)
media_effects.plot_response_curves(plot_separately=False, include_ci=False)

alt.LayerChart(...)